In [1]:
import pandas as pd

# ✅ 데이터 로드
body_type_df = pd.read_csv("data/merged_body_type.csv", encoding="utf-8")

# ✅ 체형 데이터 전처리
body_type_df["성별"] = body_type_df["성별"].map({"남성": 0, "여성": 1})

# 필요식단 키워드 추출 (One-Hot Encoding)
body_type_df["저탄수화물"] = body_type_df["필요식단"].str.contains("저탄수화물").astype(int)
body_type_df["고단백"] = body_type_df["필요식단"].str.contains("고단백").astype(int)
body_type_df["고지방"] = body_type_df["필요식단"].str.contains("고지방").astype(int)

# ✅ 필요 없는 컬럼 삭제
body_type_df.drop(columns=["필요식단"], inplace=True)

# ✅ 최종 데이터 저장
body_type_df.to_csv("processed_body_type.csv", index=False, encoding="utf-8-sig")
print("✅ processed_body_type.csv 저장 완료!")


✅ processed_body_type.csv 저장 완료!
